In [1]:
from Functionbase import *
from Backtest import *


In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()
api = sj.Shioaji(simulation=True)
api.login(
    os.getenv('API_KEY'),
    os.getenv('SECRET_KEY'),
    contracts_cb= lambda security_type: print(f"{repr(security_type)} fetch done"))

connection = sqlite3.connect('data.db')

In [ ]:
ticks = {}
kbars = {}
volume_today = {}

In [ ]:
def tbot(code):
    import pandas as pd
    
    print('--- 啟動 Tbot #{} ---'.format(code))

    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return
    
    ticks[code] = pd.concat([get_ticks(code, prev_trading_date, connection, api)[0],get_ticks(code, date, connection, api)[0]])
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()
    


In [ ]:
code = '2330'
tbot(code)

In [ ]:
ticks
ticks[code]
kbars
kbars[code]
volume_today

Callback function 用來制定如何處理接收到的ticks資料

In [5]:
@api.quote.on_quote
def quote_callback(topic: str, quote):
    
    #print('\n\ntopic:', topic)
    #print('quote:', quote)

    code = topic.split('/')[-1]

    ts = pd.to_datetime(quote.get('Date')  + ' ' + quote.get('Time'))

    if ts.hour < 9:
        return
    
    close = quote.get('Close')[0]
    volume = quote.get('Volume')[0]
    vol_sum = quote.get('VolSum')[0]

    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum




In [ ]:
@api.quote.on_quote
def quote_callback(topic: str, quote):
    
    #print('\n\ntopic:', topic)
    #print('quote:', quote)

    code = topic.split('/')[-1]

    ts = pd.to_datetime(quote.get('Date')  + ' ' + quote.get('Time'))

    if ts.hour < 9:
        return
    
    close = quote.get('Close')[0]
    volume = quote.get('Volume')[0]
    vol_sum = quote.get('VolSum')[0]

    ticks[code].loc[ts] = [close, volume]
    volume_today[code] = vol_sum

In [ ]:
api.quote.subscribe(api.Contracts.Stocks[code], quote_type='tick')

In [ ]:
api.quote.unsubscribe(api.Contracts.Stocks[code], quote_type='tick')

In [ ]:
ticks[code]
volume_today[code]

In [10]:
import time

def tbot(code):

    print('--- 啟動 Tbot #{} ---'.format(code))

    tw_calendar = get_calendar('XTAI')
    
    date = pd.to_datetime(datetime.datetime.now().date())
    prev_trading_date = tw_calendar.previous_close(date).date()

    if date not in tw_calendar.opens:
        print('今日非交易日')
        return
    
    ticks[code] = pd.concat([get_ticks(code, prev_trading_date, connection, api)[0],get_ticks(code, date, connection, api)[0]])
    ticks[code] = ticks[code][['close','volume']]
    kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
    kbars[code] = kbars[code][date:]
    volume_today[code] = kbars[code]['volume'].sum()

    api.quote.subscribe(api.Contracts.Stocks[code], quote_type='tick')

    while True:

        time.sleep(1)

        current_time = datetime.datetime.now()

        if current_time.second == 0: #秒數為0:每分鐘跑一次

            kbars[code] = get_technical_indicator(ticks_to_kbars(ticks[code], interval='1Min'))
            kbars[code] = kbars[code][date:]

            str_current_time = (current_time - datetime.timedelta(minutes=1)).strftime('%Y-%m-%d %H:%M:%S')

            try:
                currrent_kbar = kbars[code].loc[str_current_time]
            except:
                continue

            print(currrent_kbar)

    


In [ ]:
tbot('2330')